<a href="https://colab.research.google.com/github/harmeet-singh-bagga/ExplainableNLPsuite/blob/main/graph_try2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install opencv-python pytesseract transformers torch

In [3]:
!sudo apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (19.6 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

In [4]:
!pip install huggingface_hub

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [21]:
# Required Libraries
import cv2
import pytesseract
from PIL import Image
import numpy as np
from transformers import pipeline
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Step 1: Load the OCR Model for Graph Text Detection
def extract_text_from_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Preprocessing for better OCR results
    gray = cv2.medianBlur(gray, 5)
    text = pytesseract.image_to_string(gray)
    return text

# Step 2: Image Processing to Identify Graph Type
def identify_graph_type(image_path):
    image = cv2.imread(image_path)
    # Basic thresholding for edge detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Hough Line Transform to detect lines in the image (useful for line/bar charts)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)

    if lines is not None:
        return "Line Graph"

    # Future: Add more sophisticated classification methods (e.g., CNN) for other types
    return "Unknown Graph Type"

# Step 3: Hugging Face Pipeline for NLG Insights Generation
def generate_insights(text_from_graph, prompt):
    # Load pre-trained Hugging Face model for natural language generation
    model_name = "t5-base"  # You can use larger models like T5-base, etc.
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Construct the input based on the prompt and the extracted text
    input_text = f"{prompt}: {text_from_graph}"

    # Tokenize the input
    inputs = tokenizer.encode("summarize: " + input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate insights using the model
    summary_ids = model.generate(inputs, max_length=1500, min_length=100, length_penalty=0.2, num_beams=6, early_stopping=True)

    # Decode the generated response
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Step 4: Main Function to Run the Project
def analyze_graph(image_path, prompt):
    # Step 1: Extract Text from Graph
    text_from_graph = extract_text_from_image(image_path)
    print("Extracted text from graph:", text_from_graph)

    # Step 2: Identify Graph Type (For future classification expansion)
    graph_type = identify_graph_type(image_path)
    print(f"Identified Graph Type: {graph_type}")

    # Step 3: Generate Insights using NLG
    insights = generate_insights(text_from_graph, prompt)

    return insights

# Example Run
if __name__ == "__main__":
    image_path = "test_p3.png"  # Replace with the path to your graph image
    prompt = "There are some peculiar trends in the graph. Tell me some of them."

    insights = analyze_graph(image_path, prompt)
    print("Generated Insights: ", insights)


Extracted text from graph:  

8 ie Tene Fox wpm ad?

 

Identified Graph Type: Line Graph
Generated Insights:  there are some peculiar trends in the graph, tell me some of them. ie: Tene Fox wpm ad? ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie: 10 ie


In [25]:
import cv2
import numpy as np

# Graph Type Identification Function
def identify_graph_type(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Edge detection for line/bar charts
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # 1. Detect Line Graph using Hough Line Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)
    if lines is not None:
        return "Line Graph"

    # 2. Detect Bar Chart using contours (rectangular shapes)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Heuristic: bars should be taller than they are wide
        if h > w * 2:  # Adjust the ratio for specific detection criteria
            return "Bar Chart"

    # 3. Detect Pie Chart using Hough Circle Transform
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50, param1=50, param2=30, minRadius=0, maxRadius=0)
    if circles is not None:
        return "Pie Chart"

    # If no known type is detected
    return "Unknown Graph Type"

# Function to extract values from Line Graphs
def extract_values_from_line_graph(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    # Hough Line Transform for detecting line segments in the line graph
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)

    # Extract (x, y) coordinates of the detected line segments
    data_points = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        data_points.append((x1, y1))
        data_points.append((x2, y2))

    # Process these points to relate to the graph's axis (needs further axis parsing to map to real values)
    return data_points

# Function to extract values from Bar Charts
def extract_values_from_bar_chart(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    # Detect contours to identify bars
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    bar_values = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Heuristic: Bars should be taller than wide
        if h > w * 2:
            bar_values.append(h)  # Height represents value, relative to the graph’s scale

    # To map these heights to real values, you'd need to parse axis labels
    return bar_values

# Function to extract values from Pie Charts
def extract_values_from_pie_chart(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Hough Circle Transform to detect circular sections
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50, param1=50, param2=30, minRadius=0, maxRadius=0)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        sections = len(circles[0, :])
        return f"{sections} sections in Pie Chart"

    return "No sections found"

# Main function to detect the graph type and extract data
def analyze_graph(image_path):
    graph_type = identify_graph_type(image_path)
    print(f"Graph Type: {graph_type}")

    if graph_type == "Line Graph":
        data_points = extract_values_from_line_graph(image_path)
        print(f"Data points extracted from Line Graph: {data_points}")
    elif graph_type == "Bar Chart":
        bar_values = extract_values_from_bar_chart(image_path)
        print(f"Bar values extracted: {bar_values}")
    elif graph_type == "Pie Chart":
        pie_info = extract_values_from_pie_chart(image_path)
        print(f"Pie chart info: {pie_info}")
    else:
        print("Graph type is not recognized.")


Graph Type: Line Graph
Data points extracted from Line Graph: [(221, 29), (471, 29), (220, 21), (471, 21), (211, 182), (266, 296), (220, 27), (471, 27), (219, 23), (471, 23)]


In [6]:
!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 44.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash_attn


In [ ]:
import torch
from PIL import Image
import torchvision.transforms as T
from transformers import AutoProcessor, AutoModelForCausalLM
import cv2
import pytesseract
import numpy as np
import os

# Load pre-trained model from Hugging Face (Florence model)
model_name = "microsoft/Florence-2-large"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32  # Use float32 on CPU

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch_dtype, trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

# Function to preprocess the image
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size)),
        T.ToTensor(),
        # Ensure that we scale pixel values to [0, 1] range here, before applying the normalization
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=6, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # Calculate target aspect ratio
    target_ratios = sorted(
        {(i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1)},
        key=lambda x: x[0] * x[1]
    )

    # Find the closest target aspect ratio
    target_aspect_ratio = find_closest_aspect_ratio(aspect_ratio, target_ratios, orig_width, orig_height, image_size)
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)

    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)

    return processed_images

def load_image(image_file, input_size=448, max_num=6):
    if not os.path.exists(image_file):
        raise FileNotFoundError(f"The file at path '{image_file}' does not exist.")

    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

# Function to extract text using OCR
def extract_text_from_image(image_path):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"The file at path '{image_path}' does not exist.")

    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image at path '{image_path}' cannot be loaded. Check if the path is correct.")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text

# Function to generate insights from graph data
def generate_insights_from_graph(image_path, question):
    pixel_values = load_image(image_path, max_num=6).to(torch.float32 if device == "cpu" else torch.float16).to(device)

    inputs = processor(text=question, images=pixel_values, return_tensors="pt").to(device, torch_dtype)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"].to(device),
        pixel_values=inputs["pixel_values"].to(device),
        max_new_tokens=1024,
        early_stopping=True,
        do_sample=False,
        num_beams=3
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

# Function to analyze graph, extract data, and generate insights
def analyze_graph(image_path, prompt):
    # Step 1: Extract text from graph (labels, values)
    text_from_graph = extract_text_from_image(image_path)

    # Step 2: Generate deeper insights using Hugging Face models
    detailed_insights = generate_insights_from_graph(image_path, prompt)

    return detailed_insights

# Test the function with an image and a prompt
image_path = 'test_l1.png'  # Replace with actual image path
prompt = "Please describe the insights from the graph in detail."

insights = analyze_graph(image_path, prompt)
print("Generated Insights: ", insights)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
